In [1]:
import plotly.express as px
import plotly
plotly.io.templates.default = 'plotly' #'plotly_dark'
import pandas as pd
import sqlalchemy as sal

In [2]:
import pickle

In [3]:
with open("traffic_pred_horizon12.pkl", "rb") as fid:
    pred=pickle.load(fid)

In [4]:
import numpy as np
def predictions_to_df(horizon=12, timestamp="2020-06-29 22:45:00"):
    odf=pd.DataFrame(columns=["pred_speed", "pred_error"])
    
    sid_w_time=list(pred[horizon].keys())[0]
    
    tidx = np.where(pred[horizon][sid_w_time]["time"] == timestamp)[0][0]
    print(tidx)
    
    for sid in pred[horizon].keys():
        isid=int(sid)
        #print(sid)
        try:
            odf.loc[isid,"pred_speed"] = float(pred[horizon][sid].loc[tidx,"pred"])
            odf.loc[isid,"true_speed"] = float(pred[horizon][sid].loc[tidx,"truth"])
            odf.loc[isid,"pred_error"] = float(pred[horizon][sid].loc[tidx,"pred"] - pred[horizon][sid].loc[tidx,"truth"])
        except AttributeError:
            pass
    
    return odf

odf=predictions_to_df()

52085


In [5]:
def get_time_series(stid, horizon=12, day='2020-06-29'):
    sid_w_time=list(pred[horizon].keys())[0]
    tdf=pd.DataFrame(columns=["pred", "truth","time"])
    tdf["time"]=pred[horizon][sid_w_time]["time"]
    
    if str(stid) in pred[horizon].keys(): 
        tdf["pred"] = pred[horizon][str(stid)]["pred"]
        tdf["truth"] = pred[horizon][str(stid)]["truth"]

        #filter for one day - hardcoding for now
        idx=(tdf["time"] >= day) & (tdf["time"] < '2020-06-30')
    
        return tdf[idx]
    
    else:
        print('station %d not in predictions' % stid)
        return None
    
tdf=get_time_series(400017)
tdf

,pred,truth,time
51812,62.203056,62.000000,2020-06-29 00:00:00
51813,62.037960,62.000000,2020-06-29 00:05:00
51814,62.051220,62.099998,2020-06-29 00:10:00
51815,62.023640,61.900002,2020-06-29 00:15:00
51816,62.018806,61.900002,2020-06-29 00:20:00
...,...,...,...
52085,62.150963,61.700001,2020-06-29 22:45:00
52086,62.126545,61.799999,2020-06-29 22:50:00
52087,62.143738,62.000000,2020-06-29 22:55:00
52088,62.179947,62.099998,2020-06-29 23:00:00


In [6]:
tdf["time"] > '2020-05-01'

51812    True
51813    True
51814    True
51815    True
51816    True
         ... 
52085    True
52086    True
52087    True
52088    True
52089    True
Name: time, Length: 278, dtype: bool

In [8]:
host="capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
username="group3"
db="db1"
#pw=getpass.getpass("Enter database password")
pw=open("/home/daniel/Desktop/.awsdb","r").read().rstrip()

In [9]:
engine = sal.create_engine('postgresql://%s:%s@%s/%s' % (username, pw, host, db))

In [10]:
#download a snapshot of data
query="""
select l.sid, l.longitude, l.latitude, l.stype, l.fwy ,l.direc, l.abs_pm, l.state_pm
from pemslocs l
"""
sensors=pd.read_sql(query, engine, index_col="sid")
#sensors.head()

In [11]:
snap=pd.merge(sensors, odf, left_index=True, right_index=True)
#snap.head()

In [12]:
#pd.read_sql("select * from pemslocs limit 1", engine)

In [13]:
#convert from object to float
for col in ["pred_speed", "pred_error"]:
    snap[col]=snap[col].astype(float)

In [14]:
#create new column with fwy and direction as string
snap["fwdir"] = snap.apply(lambda x: str(x["fwy"]) + str(x["direc"]), axis=1)
snap["fwdir"].unique()

array(['101N', '85S', '880S', '87S', '880N', '85N', '101S', '17S', '280S',
       '237W', '237E', '87N', '17N', '280N', '680S', '680N'], dtype=object)

In [15]:
#function to move fwys in separate directions apart for visualization
def spread_lanes(sdf, fwdir, latspread=0.0005, lonspread=0.0005):
    direc=fwdir[-1]
    
    # this assumes right-hand traffic
    if direc == 'W':
         sdf["latitude"] += latspread
    elif direc == "E":
        sdf["latitude"] -= latspread
    elif direc == "S":
        sdf["longitude"] -= lonspread
    elif direc == "N":
        sdf["longitude"] += lonspread
    
    return sdf
    
#selhr=pd.read_sql("select * from usdot u where u.fwdir = '94W' order by u.abs_pm", engine)
#display(selhr.head(n=1))
#spread_lanes(selhr, '94W').head(n=1)

In [16]:
hfroads=pd.read_sql("select * from usdot", engine, index_col="usdid")
hfroads.head(n=1)

,fwdir,latitude,longitude,abs_pm
usdid,,,,
1,101N,36.917391,-121.548009,349.3


In [17]:
hrdfs=[]

from scipy.interpolate import interp1d

def interpolate_onto_hr_roads(indf, cols=["avg_speed"], dropna = False, do_spread_lanes=True):

    for fwdir in indf["fwdir"].unique():
        print("\rInterpolating average speed along %s " % fwdir, end="")
        sel=indf.query('fwdir == "%s"' % fwdir).sort_values(by="abs_pm")
        #selhr=pd.read_sql("select * from usdot u where u.fwdir = '%s' order by u.abs_pm" % fwdir, engine)
        selhr=hfroads.query("fwdir == '%s'" % fwdir).sort_values(by="abs_pm")
        
        if do_spread_lanes:
            selhr=spread_lanes(selhr, fwdir)

        if len(sel) > 1:
            #fillvalmin=sel.loc[minidx,"abs_pm"]
            #fillvalmax=sel.loc[maxidx,"abs_pm"]
            for col in cols:
                interpmod = interp1d(sel["abs_pm"].values, sel[col].values, bounds_error=False)
                selhr[col] = interpmod(selhr["abs_pm"].values)
                if dropna:
                    selhr.dropna(axis="index", how="any", subset=[col], inplace=True)
            hrdfs.append(selhr)
    print(" done.")

    selhr=pd.concat(hrdfs)
    selhr.reset_index(drop=True, inplace=True)
    
    return selhr

selhr=interpolate_onto_hr_roads(snap, cols=["pred_speed", "true_speed", "pred_error"])

Interpolating average speed along 680N  done.


In [18]:
def speed2cat(speed):
    if np.isnan(speed):
        return 'no data'
    elif speed > 65:
        return 'fast'
    elif speed > 60:
        return 'moderate'
    elif speed > 50:
        return 'slow'
    else: 
        return 'very slow'
        
selhr.head()
selhr["pred_speed_cat"]=selhr["pred_speed"].apply(speed2cat)
selhr["true_speed_cat"]=selhr["true_speed"].apply(speed2cat)

In [19]:
#selhr["pred_error"].describe()

In [20]:
def err2cat(err):
    if np.isnan(err):
        return 'no data'
    elif err > 5:
        return 'overpredict > 5 mph'
    elif err > 1:
        return 'overpredict 1 - 5 mph'
    elif err > -1: 
        return 'within 1 mph'
    elif err > -5:
        return 'underpredict 1 - 5 mph'
    else:
        return 'underpredict > 5 mph'
        
selhr["err_cat"]=selhr["pred_error"].apply(err2cat)

In [21]:
selhr.tail()

,fwdir,latitude,longitude,abs_pm,pred_speed,true_speed,pred_error,pred_speed_cat,true_speed_cat,err_cat
7905,680N,37.463662,-121.902761,9.858517,NaN,NaN,NaN,no data,no data,no data
7906,680N,37.463810,-121.902878,9.870589,NaN,NaN,NaN,no data,no data,no data
7907,680N,37.463950,-121.902988,9.881989,NaN,NaN,NaN,no data,no data,no data
7908,680N,37.464358,-121.903274,9.914249,NaN,NaN,NaN,no data,no data,no data
7909,680N,37.464540,-121.903417,9.929069,NaN,NaN,NaN,no data,no data,no data


In [22]:
a=list(selhr.columns)
a.remove("pred_error")
a

['fwdir',
 'latitude',
 'longitude',
 'abs_pm',
 'pred_speed',
 'true_speed',
 'pred_speed_cat',
 'true_speed_cat',
 'err_cat']

In [23]:
# %load road_group_split.py
def road_group_split(indf, splitcat="pred_speed_cat" ):
                     #keep_cols=["latitude", "longitude", "abs_pm", "pred_speed","fwdir"]):
    
    odict={}
    
    keep_cols=list(indf.columns)
    keep_cols.remove(splitcat)
    
    for col in keep_cols + [splitcat]:
        odict[col] = []
    #lats=[]
    #lons=[]
    #abs_pm=[]
    #cat=[]
    odict["linegroup"] = []
    
    p=0 #number holding line group
    i0=indf.index[0]
    
    for col in keep_cols + [splitcat]:
        odict[col].append(indf.loc[i0,col])
    
    #lats.append(indf.loc[i0, "latitude"])
    #lons.append(indf.loc[i0, "longitude"])
    #abs_pm.append(indf.loc[i0, "abs_pm"])
    #cat.append(indf.loc[i0, splitcat])
    #group.append(p)
    odict["linegroup"].append(p)
    
    for n in indf.index[1:]:
        if indf.loc[n,"fwdir"] != indf.loc[i0,"fwdir"]:
            p+=1
            
        if indf.loc[n,splitcat] != indf.loc[i0,splitcat] and indf.loc[n,"fwdir"] == indf.loc[i0,"fwdir"]:
            for col in keep_cols:
                odict[col].append(indf.loc[n,col])
            odict[splitcat].append(indf.loc[i0,splitcat])
            #end previous group with new point and speed cat from previous point
            #lats.append(indf.loc[n,"latitude"])
            #lons.append(indf.loc[n,"longitude"])
            #lons.append(indf.loc[n,"abs_pm"])
            #cat.append(indf.loc[i0,splitcat])
            #group.append(p)
            odict["linegroup"].append(p)
            p+=1
        #start new group
        for col in keep_cols + [splitcat]:
            odict[col].append(indf.loc[n,col])
        #lats.append(indf.loc[n,"latitude"])
        #lons.append(indf.loc[n,"longitude"])
        #cat.append(indf.loc[n,splitcat])
        #group.append(p)        
        odict["linegroup"].append(p)
        i0=n
    
    #odf=pd.DataFrame({"latitude": lats, "longitude": lons, splitcat: cat, "linegroup": group})
    odf=pd.DataFrame(odict)
    return odf

selhrg=road_group_split(selhr)
#selhrg.tail()

In [24]:
cdm={"fast": "green",
     "moderate": "orange",
     "slow": "red",
     "very slow": "brown",
     "no data": "gray"}

In [25]:
selhrg.head(n=1)

,fwdir,latitude,longitude,abs_pm,pred_speed,true_speed,pred_error,true_speed_cat,err_cat,pred_speed_cat,linegroup
0,101N,36.917391,-121.547509,349.3,NaN,NaN,NaN,no data,no data,no data,0


In [26]:
cdme={"within 1 mph": "green",
     "underpredict 1 - 5 mph": "orange",
     "underpredict > 5 mph": "red",
     "overpredict 1 - 5 mph": "lightgreen",
     "overpredict > 5 mph": "darkgreen",
     "no data": "gray"}

In [27]:
#fig.write_image("traffic_layer_bay_pred_error.jpg")

In [28]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options

app.layout = html.Div(children=[
    html.H1(children='Traffic Dashboard'),

    html.Div(children='''
        DRCNN Network Performance
    '''),
    
     html.H6('Variable'),
     dcc.RadioItems(id='plottype',
                   options=[{"label":"True traffic","value": "true_speed_cat"},
                           {"label":"Predicted traffic", "value": "pred_speed_cat"},
                           {"label":"Prediction error", "value": "err_cat"}],
                   value = 'pred_speed_cat',
                   labelStyle={'display': 'block'}),

     html.Div(children=[
         dcc.Graph(id='roadmap', style={'display': 'inline-block'}),
     #html.H6('Time Series'),
         dcc.Graph(id='timeseries', style={'display': 'inline-block'})]),
     html.Pre(id='click-data')
])

In [29]:
@app.callback(
Output('roadmap','figure'),
Input('plottype', 'value'))
def plot_roads(plotcat):
    plotly.io.templates.default='plotly'
    #del fig
    
    print(plotcat)
    selhrg=road_group_split(selhr, splitcat=plotcat) 
                        #keep_cols=keepcols)

    fig = px.line_mapbox(
    #fig=px.scatter_mapbox
        selhrg, lat="latitude", lon="longitude", color=plotcat,
                            #mapbox_style="carto-darkmatter",
                            mapbox_style="carto-positron",
                            #color_continuous_scale="aggrnyl" ,
                            line_group = "linegroup",
                            hover_data=None, #list(selhrg.columns),
                            center = {"lat": 37.34, "lon": -121.93},
                            color_discrete_map= cdme if plotcat == "err_cat" else cdm,
                            zoom=10,
                            height=600, width=600)

    for dat in fig.data:
        dat["line"]["width"] = 5

    sensors["size"] = 3
    fig2 = px.scatter_mapbox(sensors.reset_index(), lat="latitude", lon="longitude",
                            #hover_data=["stype","fwy","direc","abs_pm","pred_speed"],
                            hover_name="sid",
                            hover_data=list(sensors.reset_index().columns),
                            mapbox_style="stamen-terrain",
                             color="stype",
                            color_discrete_map={"ML": "blue"},
                            size="size",
                            opacity=0.35,
                            size_max=5)
    #for dat in fig2.data:
    fig.add_trace(fig2.data[0])
    
    fig.update_layout(uirevision=True)

    return  fig
    #px.scatter(sel, x="longitude", y="latitude", color="total_flow").update_traces(mode='lines+markers')

In [30]:
#plot_roads("true_speed_cat")
#def plot_time_series(clickData):

In [37]:
import plotly.graph_objects as go

In [47]:
@app.callback(
Output('timeseries','figure'),
Input('roadmap','clickData'))
def plot_time_series(clickData):
#def plot_time_series(stid):
    if type(clickData) == type(None):
        return None
    
    if "hovertext" not in clickData["points"][0].keys():
        print("Click directly onto a sensor to plot time series.")
        return None

    stid=clickData["points"][0]["hovertext"]
    print(stid)
    tdf=get_time_series(stid)

    if type(tdf) == type(None):
        print("got None time series for %d" % stid)
        return None
    
    display(tdf.head(n=3))

    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x=tdf["time"], y=tdf["pred"], mode='lines', name='prediction'))
    fig3.add_trace(go.Scatter(x=tdf["time"], y=tdf["truth"], mode='lines', name='observation'))
    fig3.write_image("test_output.jpg")

    return fig3

In [42]:
#plot_time_series(400837)

In [48]:
import json
@app.callback(
    Output('click-data', 'children'),
    Input('roadmap', 'clickData'))
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)

In [49]:
#plot_roads("true_speed_cat")

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/May/2021 00:23:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2021 00:23:13] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2021 00:23:13] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2021 00:23:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2021 00:23:13] "POST /_dash-update-component HTTP/1.1" 200 -


pred_speed_cat


127.0.0.1 - - [28/May/2021 00:23:15] "POST /_dash-update-component HTTP/1.1" 200 -


err_cat


127.0.0.1 - - [28/May/2021 00:23:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2021 00:25:10] "POST /_dash-update-component HTTP/1.1" 200 -


400508


,pred,truth,time
51812,64.425667,64.500000,2020-06-29 00:00:00
51813,64.585579,64.599998,2020-06-29 00:05:00
51814,64.457184,64.400002,2020-06-29 00:10:00


127.0.0.1 - - [28/May/2021 00:25:12] "POST /_dash-update-component HTTP/1.1" 200 -
